## Selecting a model

We are facing a regression problem. Since interactions between features are to be expected, a decision-tree based approach seems appropriate, but let's also try a Lasso regression and SVR to see what happens.

In [2]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

# import my method from the source code
%aimport features.build_features
%aimport visualization.visualize
from features.build_features import read_raw_data
import numpy as np
import pandas as pd

In [3]:
# features selected by RFECV in 3-fm-feature-reduction 

features = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'BsmtFullBath', 'KitchenAbvGr', 'KitchenQual', 'FireplaceQu',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'ScreenPorch', 'PoolArea', 'MoSold', 'MSSubClass_60',
       'MSZoning_C (all)', 'LotConfig_CulDSac', 'Neighborhood_ClearCr',
       'Neighborhood_Crawfor', 'Neighborhood_OldTown', 'Neighborhood_StoneBr',
       'Condition1_Artery', 'Condition1_Norm', 'Exterior1st_BrkFace',
       'BsmtFinType1_GLQ', 'Functional_Typ', 'GarageType_Attchd',
       'SaleType_New', 'SaleCondition_Abnorml', 'SaleCondition_Family',
       'BsmtFinSF']

fid = features + ['Id']

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


df = read_raw_data("../data/intermediate/eval.csv")

X_w_id = df[fid]
X = df[features]
y = df['SalePrice'] 

# use log transform to ensure positive predictions with lasso!
y_log = np.log1p(y)


pipe = make_pipeline(Lasso(max_iter=50000))
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(pipe, X, y_log, cv=shuffle_split, scoring='neg_mean_squared_error')
scores = [sqrt(abs(s)) for s in scores]
print(scores)
print("Lasso: {}\n".format(np.mean(scores)))

pipe = make_pipeline(MinMaxScaler(), SVR(C=1000))
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(pipe, X, y_log, cv=shuffle_split, scoring='neg_mean_squared_error')
scores = [sqrt(abs(s)) for s in scores]
print(scores)
print("SVR: {}\n".format(np.mean(scores)))

pipe = make_pipeline(GradientBoostingRegressor())
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(pipe, X, y, cv=shuffle_split, scoring='neg_mean_squared_log_error')
scores = [sqrt(abs(s)) for s in scores]
print(scores)
print("GBR: {}\n".format(np.mean(scores)))


[0.22567824583081775, 0.21553487753850015, 0.21058137245633135, 0.18247887895216577, 0.19169848844497978, 0.19677458508357587, 0.17090873525538192, 0.18797073238340353, 0.18185491116691205, 0.23037204141204656]
Lasso: 0.19938528685241147

[0.1565447739716441, 0.15492588629000068, 0.15451953233726023, 0.15158406010047398, 0.1453904136927294, 0.14950359155857088, 0.1540218229360688, 0.15108365154117273, 0.15045649961723584, 0.14257484285861594]
SVR: 0.15106050749037725

[0.13634252999333954, 0.1366370958085369, 0.14997087184780028, 0.14223500492996774, 0.14363685282383046, 0.14258143739142887, 0.1306995758863363, 0.13384319761885227, 0.13347759284265323, 0.13814968693116458]
GBR: 0.138757384607391



The gradient boosting regressor with default parameters already achieves the best results. The parameters for the SVR and Lasso have been fixed by hand with trial and error. A more rigorous approach would of course be recommended. Nevertheless let's continue with the gradient boosting tree since it produces the best results.

What parameters should be selected for the GBR?

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X_w_id, y)

param_grid = {'max_depth' : [3, 4, 5],
              'learning_rate' : [0.01, 0.1],
              'min_samples_leaf' : [1, 2]}

gridcv = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid,    
                            cv=5, scoring='neg_mean_squared_log_error', verbose=1)
gridcv.fit(X_train, y_train)


print("Best cross_validaton score : {:.3f}".format(sqrt(abs(gridcv.best_score_))))
print("Test set score : {:.3f}".format(sqrt(abs(gridcv.score(X_test, y_test)))))
print("Best parameters : {}".format(gridcv.best_params_))


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   17.4s finished


Best cross_validaton score : 0.138
Test set score : 0.135
Best parameters : {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 1}


A maximum depth of 4 levels, the default learning rate and default number of leaf nodes produce the best results. Let's take a look at the predictions the GBR makes on the test set.

In [6]:
from sklearn.metrics import mean_squared_log_error
gbr = GradientBoostingRegressor(learning_rate=0.1, max_depth=4, min_samples_leaf=2)
gbr.fit(X_train, y_train)
prediction = gbr.predict(X_test)

for t,p in zip(y_test, prediction):
    print("truth: {}, predict.: {}, score: {}".format(t, p, sqrt(mean_squared_log_error([t], [p]))))


truth: 149700, predict.: 156789.53464498225, score: 0.04627076887967796
truth: 139000, predict.: 112359.17034869747, score: 0.21277160901718162
truth: 140000, predict.: 132809.89428047783, score: 0.05272329651874763
truth: 76500, predict.: 89243.10610740425, score: 0.15407156782438847
truth: 212000, predict.: 220446.53052078566, score: 0.039068718106829436
truth: 68400, predict.: 74000.6453792225, score: 0.0786998834066619
truth: 224900, predict.: 206486.75702517046, score: 0.08541918274623583
truth: 116000, predict.: 107719.26453266211, score: 0.07406108807523637
truth: 188000, predict.: 145113.63152845812, score: 0.2589232896086848
truth: 129000, predict.: 181507.9099374158, score: 0.34148458679466565
truth: 208900, predict.: 214085.87471989953, score: 0.024521433748315502
truth: 181000, predict.: 175486.5946241146, score: 0.03093420160736926
truth: 187100, predict.: 195462.3082392804, score: 0.04372410216931044
truth: 250000, predict.: 222262.37877315463, score: 0.1176018483174186
t

truth: 177000, predict.: 180594.30448243636, score: 0.020103258825965753
truth: 164700, predict.: 159773.29441113956, score: 0.030369549424559494
truth: 143000, predict.: 142500.51052256956, score: 0.003499023431542625
truth: 113000, predict.: 112592.25741878276, score: 0.0036148351459051042
truth: 129500, predict.: 125782.27946842657, score: 0.029128181290468902
truth: 275000, predict.: 270429.8964837677, score: 0.016758134238097355
truth: 179400, predict.: 165648.99261154313, score: 0.07974643960208283
truth: 94750, predict.: 124318.44516299381, score: 0.27160202563819524
truth: 170000, predict.: 172910.0437379324, score: 0.016972944820091485
truth: 111250, predict.: 126354.65123242677, score: 0.12731164987386023
truth: 144000, predict.: 154196.01706849405, score: 0.06841087238349353
truth: 87000, predict.: 109579.86626234732, score: 0.23074316853373134
truth: 166000, predict.: 122618.23214693775, score: 0.302909932173856
truth: 106000, predict.: 97819.99924799844, score: 0.080309257

We see that the score is quite close to 0 for most records, however it is very high for others. Let's visualize the performance per record on original features vs SalePrice plots.
Each dot represents a record, the color of the dot indicates the score, the darker the dot the worse the prediction.

In [9]:
original = read_raw_data("../data/raw/train.csv")
result = pd.DataFrame(X_test)
result = result[['Id']]
result = pd.merge(result, original, on='Id', how='inner')
result['Prediction'] = prediction

def msle(row):
    return sqrt(mean_squared_log_error([row['SalePrice']], [row['Prediction']]))

result['Score'] = result.apply(msle, axis=1)
result.to_csv("results.csv")
from IPython.display import HTML
HTML('<iframe src=../reports/figures/score-gbr.pdf width=700 height=350></iframe>')

The main observation is that the worst predictions seem to preliminarily concern either low-price houses or outliers. We see that
- MSSubClass : many bad scores obtained for classes 20 and 30 (category not included in those retained with RFECV)
- OverallQual : bad quality --> bad score (bad quality --> lower SalePrice)
- GarageArea : no garage --> bad score

Let's confirm

In [10]:
bins = np.arange(np.amin(result['SalePrice']), np.amax(result['SalePrice']), step=50000)
print(bins)
result['BinPrice'] = np.digitize(result['SalePrice'], bins)

freq_score = result[(result['Score'] >= 0.1)]['BinPrice'].value_counts()/result['BinPrice'].value_counts()
freq_price = result['BinPrice'].value_counts()/len(result)
print("relative frequency of bad predictions per SalePrice: \n{}".format(freq_score))
print("relative frequency of bad predictions per SalePrice, weighted by relative frequency of SalePrice: \n{}"
      .format(freq_score*freq_price))

[ 34900  84900 134900 184900 234900 284900 334900 384900 434900 484900
 534900 584900 634900 684900 734900]
relative frequency of bad predictions per SalePrice: 
1     0.857143
2     0.287129
3     0.216667
4     0.215385
5     0.473684
6     0.307692
7     0.666667
8     0.333333
9     1.000000
15    1.000000
Name: BinPrice, dtype: float64
relative frequency of bad predictions per SalePrice, weighted by relative frequency of SalePrice: 
1     0.032877
2     0.079452
3     0.071233
4     0.038356
5     0.049315
6     0.010959
7     0.010959
8     0.005479
9     0.002740
15    0.002740
Name: BinPrice, dtype: float64


We see that for houses with a price between 39.300 and 89.300 the majority of predictions that are made are bad, The same holds for houses lying in the price range 239.300 - 289.300. If we weigh this with the relative frequency of the price ranges, we see that it is for houses in the price range 89.300 - 139.300 for which most bad predictions are made.

In [11]:
# relative frequency of bad scores per MSSubClass category, weighted by relative frequency of that category
(result[(result['Score'] >= 0.1)]['MSSubClass'].value_counts()/result['MSSubClass'].value_counts())*(result['MSSubClass'].value_counts()/len(result))

20     0.084932
30     0.013699
40     0.002740
45     0.002740
50     0.041096
60     0.054795
70     0.016438
75     0.008219
80     0.019178
85          NaN
90     0.016438
120    0.016438
160    0.016438
180    0.002740
190    0.008219
Name: MSSubClass, dtype: float64

Performance for class 20 is worse than for other classes, however it it worse for classes 50 and 60 than for 30.

In [12]:
freq_score = result[(result['Score'] >= 0.1)]['OverallQual'].value_counts()/result['OverallQual'].value_counts()
freq_price = result['OverallQual'].value_counts()/len(result)
print("relative frequency of bad predictions per overall quality: \n{}".format(freq_score))
print("relative frequency of bad predictions per overall quality, weighted by relative frequency of quality rating: \n{}"
      .format(freq_score*freq_price))

relative frequency of bad predictions per overall quality: 
2     1.000000
3     0.166667
4     0.472222
5     0.230769
6     0.237113
7     0.362500
8     0.295455
9     0.428571
10    1.000000
Name: OverallQual, dtype: float64
relative frequency of bad predictions per overall quality, weighted by relative frequency of quality rating: 
2     0.002740
3     0.002740
4     0.046575
5     0.057534
6     0.063014
7     0.079452
8     0.035616
9     0.008219
10    0.008219
Name: OverallQual, dtype: float64


Bad scores are obtained for houses with a quality ranking of 1, 10 and 3. It is however the houses in classes 6 and 5 for which most predictions are made.

Let's see how other models perform on the test data.

In [7]:
from sklearn.linear_model import Lasso

y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

lasso = Lasso(max_iter=50000)
lasso.fit(X_train, y_train_log)
lasso_predict = lasso.predict(X_test)

for t,tl,p,l in zip(y_test, y_test_log, prediction, lasso_predict):
    score_gbr = sqrt(mean_squared_log_error([t], [p]))
    if(score_gbr > 0.1):
        print("score-gbr: {}, score-lasso: {}".format(score_gbr, sqrt(mean_squared_error([tl], [l]))))


score-gbr: 0.21277160901718162, score-lasso: 0.0871720136012204
score-gbr: 0.15407156782438847, score-lasso: 0.28730848853702007
score-gbr: 0.2589232896086848, score-lasso: 0.08178458303863678
score-gbr: 0.34148458679466565, score-lasso: 0.5743351878905898
score-gbr: 0.1176018483174186, score-lasso: 0.07605277216723749
score-gbr: 0.20333510457363246, score-lasso: 0.12816572767591516
score-gbr: 0.11550920885772342, score-lasso: 0.19517526037136435
score-gbr: 0.2363493105266734, score-lasso: 0.17994923966578646
score-gbr: 0.12474962284233371, score-lasso: 0.05696186319185159
score-gbr: 0.11655547993473725, score-lasso: 0.04259919484794139
score-gbr: 0.10998271733197917, score-lasso: 0.4587709439794647
score-gbr: 0.1256339554008985, score-lasso: 0.17762910413095767
score-gbr: 0.14657613665691116, score-lasso: 0.00016118876914461566
score-gbr: 0.15548728175247994, score-lasso: 0.1280962145618112
score-gbr: 0.23028527533670662, score-lasso: 0.3095977704121182
score-gbr: 0.21849267936937267,

We see that for some of the houses for which the GBR produced a bad score the prediction is improved by Lasso. Let's try combining Lasso and GBR.

In [8]:
import numpy as np
avg_score = 0
for t, gbrp, lp in zip(y_test, prediction, lasso_predict):
    mean_pred = (np.expm1(lp) + gbrp)/2.0
    score = sqrt(mean_squared_log_error([t], [mean_pred]))
    avg_score += score
avg_score = avg_score/float(len(lasso_predict))
print(avg_score)

0.10025812686173259


Combining both models actually allows to improve the overall score. The best combination on the Kaggle test set is 20% Lasso and 80% GBR : using the features selected previously and this averaging of models we get a score of 0.12916.

Let's, for fun, see what happens if we add an SVM on top. SVMs are known to be sensitive to the scaling of the data, so we should apply a scaler, 

In [9]:
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)

svr_rbf = SVR(kernel='linear', C=1000)
svr_rbf.fit(X_train_minmax, y_train_log)
svr_pred = svr_rbf.predict(X_test_minmax)

for t, lp, gbrp, svr in zip(y_test, lasso_predict, prediction, svr_pred):
#    print("gbr: {:.3f}, lasso: {:.3f}, svr: {:.3f}, real: {}".format(gbrp, np.expm1(lp), np.expm1(svr), t))
    mean_pred = (np.expm1(lp) + gbrp + np.expm1(svr))/3.0
    score = sqrt(mean_squared_log_error([t], [mean_pred]))
    avg_score += score
avg_score = avg_score/float(len(X_test))
print(avg_score)

0.09262488260582817


Adding the SVR actually improves the score on the test set. Since we saw that the GBR performs rather bad on the more extreme feature values, using KNeighbors should allow to mitigate this. Let's see what happens when we replace the SVR with KNeighbors.

In [10]:
from sklearn.neighbors import KNeighborsRegressor

knb = KNeighborsRegressor(n_neighbors=5)
knb.fit(X_train, y_train)
nn_predict = knb.predict(X_test)

for t, gbrp, lp, n in zip(y_test, prediction, lasso_predict, nn_predict):
    mean_pred = (np.expm1(lp) + gbrp + n)/3.0
    score = sqrt(mean_squared_log_error([t], [mean_pred]))
#    print("gbr: {:.3f}, lasso: {:.3f}, nn: {:.3f}, mean: {:.3f}, real: {}, score: {}".format(gbrp, np.expm1(lp), n, mean_pred, t, score))
    avg_score += score
avg_score = avg_score/float(len(nn_predict))
print(avg_score)

0.11856610534011609


KNeighbors does not improve the results.

Now that we have seen that we can improve the quality of the prediction on the test set by combining several models, let's build a transformer combining the 4 models and let a LinearRegression determine the final prediction. While it would be interesting to do a GridSearch to set the parameters of the 4 models this will be too time-consuming at this stage. Let's just run a GridSearch to determine which of the 4 models should actually be used.

In [35]:
%aimport models.Stacking_Transformer
import models.Stacking_Transformer as st
from sklearn.utils.estimator_checks import check_estimator

# Check we have an estimator adhereing to the scikit-learn interface and standards
check_estimator(st.StackingTransformer)

In [34]:
%aimport models.Stacking_Transformer
import models.Stacking_Transformer as st
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn. model_selection import GridSearchCV
from sklearn.utils.estimator_checks import check_estimator

param_grid = {"transform__dogbr": [True],
              "transform__dolasso": [False, True],
              "transform__dosvr" : [False, True],
              "transform__doknn" : [False, True]
             }

pipe = Pipeline([("transform", st.StackingTransformer()), ("lr", LinearRegression())])
gridcv = GridSearchCV(pipe, cv=5, scoring='neg_mean_squared_log_error', param_grid=param_grid, verbose=100)
gridcv.fit(X_train, y_train)

print("Best cross_validaton score : {:.3f}".format(sqrt(abs(gridcv.best_score_))))
print("Test set score : {:.3f}".format(sqrt(abs(gridcv.score(X_test, y_test)))))
print("Best parameters : {}".format(gridcv.best_params_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] transform__dogbr=True, transform__doknn=False, transform__dolasso=False, transform__dosvr=False 
[CV]  transform__dogbr=True, transform__doknn=False, transform__dolasso=False, transform__dosvr=False, score=-0.015076082546312129, total=   0.3s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[CV] transform__dogbr=True, transform__doknn=False, transform__dolasso=False, transform__dosvr=False 
[CV]  transform__dogbr=True, transform__doknn=False, transform__dolasso=False, transform__dosvr=False, score=-0.0254929031957322, total=   0.3s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[CV] transform__dogbr=True, transform__doknn=False, transform__dolasso=False, transform__dosvr=False 
[CV]  transform__dogbr=True, transform__doknn=False, transform__dolasso=False, transform__dosvr=False, score=-0.016616802178356857, total=   0.3s
[Parallel(n_jobs=1)]: Done   3 out of 

[CV]  transform__dogbr=True, transform__doknn=True, transform__dolasso=False, transform__dosvr=True, score=-0.015188291460591408, total=  25.2s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:  5.0min remaining:    0.0s
[CV] transform__dogbr=True, transform__doknn=True, transform__dolasso=False, transform__dosvr=True 
[CV]  transform__dogbr=True, transform__doknn=True, transform__dolasso=False, transform__dosvr=True, score=-0.023082400582199614, total=  25.3s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  5.4min remaining:    0.0s
[CV] transform__dogbr=True, transform__doknn=True, transform__dolasso=False, transform__dosvr=True 
[CV]  transform__dogbr=True, transform__doknn=True, transform__dolasso=False, transform__dosvr=True, score=-0.015703491997878957, total=  26.9s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:  5.9min remaining:    0.0s
[CV] transform__dogbr=True, transform__doknn=True, transform__dolasso=False, transform__dosvr=True 
[CV]  transform__dogbr=True,

Further attempts at improving the model could be made by :

- feature engineering : PCA + polynomials
- grid searching over the models to use
- possibly integrating original features in the transformer result and doing a GBR over them - the GBR could learn in which cases which model performs better
- using XGBoost library for the GradientBoosting -> should allow to increment the number of estimators without increasing running time (danger: overfitting)
